# Stock NeurIPS2018 Part 1. Data
This series is a reproduction of paper *the process in the paper Practical Deep Reinforcement Learning Approach for Stock Trading*. 

This is the first part of the NeurIPS2018 series, introducing how to use FinRL to fetch and process data that we need for ML/RL trading.

Other demos can be found at the repo of [FinRL-Tutorials]((https://github.com/AI4Finance-Foundation/FinRL-Tutorials)).

# Part 1. Install Packages

In [1]:
## install finrl library
# !pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

In [4]:
import pandas as pd
import numpy as np
import datetime
import yfinance as yf

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl import config_tickers
from finrl.config import INDICATORS

import itertools

# Part 2. Fetch data

[yfinance](https://github.com/ranaroussi/yfinance) is an open-source library that provides APIs fetching historical data form Yahoo Finance. In FinRL, we have a class called [YahooDownloader](https://github.com/AI4Finance-Foundation/FinRL/blob/master/finrl/meta/preprocessor/yahoodownloader.py) that use yfinance to fetch data from Yahoo Finance.

**OHLCV**: Data downloaded are in the form of OHLCV, corresponding to **open, high, low, close, volume,** respectively. OHLCV is important because they contain most of numerical information of a stock in time series. From OHLCV, traders can get further judgement and prediction like the momentum, people's interest, market trends, etc.

## Data for a single ticker

Here we provide two ways to fetch data with single ticker, let's take Apple Inc. (AAPL) as an example.

### Using yfinance

In [3]:
aapl_df_yf = yf.download(tickers = "aapl", start='2020-01-01', end='2020-01-31')

[*********************100%%**********************]  1 of 1 completed


In [4]:
aapl_df_yf.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02,74.059998,75.150002,73.797501,75.087502,72.960449,135480400
2020-01-03,74.287498,75.144997,74.125000,74.357498,72.251144,146322800
2020-01-06,73.447502,74.989998,73.187500,74.949997,72.826843,118387200
2020-01-07,74.959999,75.224998,74.370003,74.597504,72.484352,108872000
2020-01-08,74.290001,76.110001,74.290001,75.797501,73.650360,132079200


### Using FinRL

In FinRL's YahooDownloader, we modified the data frame to the form that convenient for further data processing process. We use adjusted close price instead of close price, and add a column representing the day of a week (0-4 corresponding to Monday-Friday).

In [31]:
aapl_df_finrl = YahooDownloader(start_date = '2020-01-01',
                                end_date = '2020-01-31',
                                ticker_list = ['aapl']).fetch_data()

[*********************100%%**********************]  1 of 1 completed

Shape of DataFrame:  (20, 8)


In [34]:
aapl_df_finrl[:20]

,date,open,high,low,close,volume,tic,day
0,2020-01-02,74.059998,75.150002,73.797501,72.960464,135480400,aapl,3
1,2020-01-03,74.287498,75.144997,74.125000,72.251137,146322800,aapl,4
2,2020-01-06,73.447502,74.989998,73.187500,72.826851,118387200,aapl,0
3,2020-01-07,74.959999,75.224998,74.370003,72.484337,108872000,aapl,1
4,2020-01-08,74.290001,76.110001,74.290001,73.650352,132079200,aapl,2
5,2020-01-09,76.809998,77.607498,76.550003,75.214737,170108400,aapl,3
6,2020-01-10,77.650002,78.167503,77.062500,75.384781,140644800,aapl,4
7,2020-01-13,77.910004,79.267502,77.787498,76.995323,121532000,aapl,0
8,2020-01-14,79.175003,79.392502,78.042503,75.955643,161954400,aapl,1
9,2020-01-15,77.962502,78.875000,77.387497,75.630135,121923600,aapl,2


## Data for the chosen tickers

In [7]:
config_tickers.DOW_30_TICKER

['AXP',
 'AMGN',
 'AAPL',
 'BA',
 'CAT',
 'CSCO',
 'CVX',
 'GS',
 'HD',
 'HON',
 'IBM',
 'INTC',
 'JNJ',
 'KO',
 'JPM',
 'MCD',
 'MMM',
 'MRK',
 'MSFT',
 'NKE',
 'PG',
 'TRV',
 'UNH',
 'CRM',
 'VZ',
 'V',
 'WBA',
 'WMT',
 'DIS',
 'DOW']

# Original data

In [35]:
df_raw = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = ['VIC.VN']).fetch_data()

[*********************100%%**********************]  1 of 1 completed

Shape of DataFrame:  (203, 8)


In [36]:
df_raw[:20]

,date,open,high,low,close,volume,tic,day
0,2023-07-17,51600.0,53100.0,51400.0,52900.0,12105500,VIC.VN,0
1,2023-07-19,52900.0,52900.0,52100.0,52100.0,1816000,VIC.VN,2
2,2023-07-20,52100.0,52600.0,51600.0,52000.0,1893807,VIC.VN,3
3,2023-07-21,52000.0,52300.0,51600.0,52100.0,2324811,VIC.VN,4
4,2023-07-24,52200.0,52600.0,51800.0,52000.0,2611616,VIC.VN,0
5,2023-07-26,51400.0,51900.0,51000.0,51000.0,4119713,VIC.VN,2
6,2023-07-28,51100.0,51600.0,50900.0,51500.0,3550220,VIC.VN,4
7,2023-08-03,58300.0,59500.0,58000.0,58200.0,7522816,VIC.VN,3
8,2023-08-04,59000.0,62200.0,59000.0,62200.0,21210810,VIC.VN,4
9,2023-08-07,63500.0,64300.0,62400.0,63900.0,8530327,VIC.VN,0


# OUR CLASS

In [10]:
TRAIN_START_DATE = '2015-01-05'
TRAIN_END_DATE = '2022-07-01'
TRADE_START_DATE = '2022-07-01'
TRADE_END_DATE = '2024-05-23'

In [11]:
class OurDatasetCollector:
    def __init__(self):
        pass

    def check_dataset(self, df):
        duplicates = df.duplicated().sum()
        print(f"Number of duplicated rows: {duplicates}")

        missing_values = df.isnull().sum().sum()
        print(f"Number of missing values: {missing_values}")

        day_counts = df['day'].value_counts().sort_index()
        print("Day of the week counts:")
        for day, count in day_counts.items():
            print(f"{day}: {count}")

    def csv_to_yf(self, csv_path, stock_ticker):

        df = pd.read_csv(csv_path)
        
        df['date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y')
        df['open'] = df['Open'].str.replace(',', '').astype(float)
        df['high'] = df['High'].str.replace(',', '').astype(float)
        df['low'] = df['Low'].str.replace(',', '').astype(float)
        df['close'] = df['Price'].str.replace(',', '').astype(float)    
        df['volume'] = df['Vol.'].apply(lambda x: float(x.replace('M', '').replace('K', '')) * (1e6 if 'M' in x else 1e3 if 'K' in x else 1))
        df['tic'] = stock_ticker
        df['day'] = df['date'].dt.dayofweek
        df['date'] = df['date'].astype(str)
        
        drop_col = ['Date',	'Price', 'Open', 'High', 'Low',	'Vol.','Change %']
        for col in drop_col:
            df.drop(columns=col, inplace=True)
        return df
    
    def Importer(self, csv_path, tik):
        df = self.csv_to_yf(csv_path, tik)
        self.check_dataset(df)
        return df


In [12]:
collector = OurDatasetCollector()
vic_df = collector.Importer('VIC Historical Data.csv', 'vic.vn')
print(vic_df.dtypes)

Number of duplicated rows: 0
Number of missing values: 0
Day of the week counts:
0: 458
1: 470
2: 470
3: 473
4: 470
date       object
open      float64
high      float64
low       float64
close     float64
volume    float64
tic        object
day         int32
dtype: object


# Part 3: Preprocess Data
We need to check for missing data and do feature engineering to convert the data point into a state.
* **Adding technical indicators**. In practical trading, various information needs to be taken into account, such as historical prices, current holding shares, technical indicators, etc. Here, we demonstrate two trend-following technical indicators: MACD and RSI.
* **Adding turbulence index**. Risk-aversion reflects whether an investor prefers to protect the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the turbulence index that measures extreme fluctuation of asset price.

Hear let's take **MACD** as an example. Moving average convergence/divergence (MACD) is one of the most commonly used indicator showing bull and bear market. Its calculation is based on EMA (Exponential Moving Average indicator, measuring trend direction over a period of time.)

In [16]:
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS,
                     use_vix=True,
                     use_turbulence=True,
                     user_defined_feature = False)

processed = fe.preprocess_data(vic_df)

[*********************100%%**********************]  1 of 1 completed

Successfully added technical indicators
Shape of DataFrame:  (2362, 8)
date     object
vix     float64
dtype: object
Successfully added vix


Successfully added turbulence index


In [17]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [19]:
processed_full.tail()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
3419,2024-05-16,vic.vn,46150.0,48200.0,45600.0,46750.0,2780000.0,3.0,99.898776,48153.826791,40866.173209,53.841753,53.353854,4.943208,45563.333333,45793.333333,12.42,0.141965
3420,2024-05-17,vic.vn,46750.0,47000.0,46250.0,46850.0,2170000.0,4.0,206.776164,48125.304235,40879.695765,54.086923,49.514837,4.943208,45530.000000,45790.833333,11.99,0.008705
3423,2024-05-20,vic.vn,47000.0,48900.0,45900.0,46700.0,3220000.0,0.0,276.189775,48175.695557,40859.304443,53.644783,68.158565,2.641157,45510.000000,45760.833333,12.15,0.017202
3424,2024-05-21,vic.vn,43450.0,47700.0,43450.0,46450.0,3340000.0,1.0,307.483176,48340.937021,40834.062979,52.899214,19.399960,6.383923,45466.666667,45747.500000,11.86,0.051082
3425,2024-05-22,vic.vn,43200.0,47000.0,43200.0,47000.0,3150000.0,2.0,372.371360,48586.148463,41028.851537,54.343358,16.991240,7.244847,45438.333333,45737.500000,12.29,0.263863


# Part 4: Save the Data

### Split the data for training and trading

In [20]:
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

1813
456


### Save data to csv file

For Colab users, you can open the virtual directory in colab and manually download the files.

For users running on your local environment, the csv files should be at the same directory of this notebook.

In [21]:
train.to_csv('train_data.csv')
trade.to_csv('trade_data.csv')